In [136]:
import pandas as pd
import numpy as np
from typing import List, Optional

import statsmodels.api as sm

# plt.style.use('dark_background')
random_state=42

In [137]:
tesis = pd.read_csv("../Datos/tesis_final.csv")
tesis["Adherencia_Perfecta"] = tesis["Adherencia_Total"] == 1
tesis.head()

,idPaciente,Mes,TAS,Adherencia,Peso,Altura,IMC,DBT,Sexo,Edad,Fuma,ant_HTA,tas_basal,ICC,Adherencia_Acumulada,Adherencia_Total,TAS_Media_Acumulada,Adherencia_Perfecta
0,4026,1,119.000000,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,119.000000,True
1,4026,2,127.000000,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,123.000000,True
2,4026,3,140.000000,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,128.666667,True
3,4026,4,146.712710,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,133.178178,True
4,4026,5,177.708084,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,142.084159,True


In [138]:
# tesis["Mes"] = tesis["Mes"] - 1

### Imputar valores faltantes

In [139]:
categorical = ["Adherencia", "Sexo", "Adherencia_Perfecta"]
numerical = ["Mes", "TAS", "Edad", "tas_basal", "Adherencia_Acumulada", "Adherencia_Total"]

In [140]:
tesis = tesis[categorical + numerical + ["idPaciente"]]

In [141]:
from statistics import mean, median, mode

for cat in categorical:
    tesis[cat].fillna(mode(tesis[tesis["Mes"] == 1][cat]), inplace=True)

for num in numerical:
    tesis[num].fillna(np.mean(tesis[tesis["Mes"] == 1][num]), inplace=True)

In [142]:
tesis["Adherencia_lag1"] = tesis.groupby("idPaciente")["Adherencia"].shift(1).fillna(0)
tesis["Adherencia_lag2"] = tesis.groupby("idPaciente")["Adherencia"].shift(2).fillna(0)
tesis["Adherencia_Acumulada_lag1"] = tesis.groupby("idPaciente")["Adherencia_Acumulada"].shift(1).fillna(0)
tesis["TAS_lag1"] = tesis.groupby("idPaciente")["TAS"].shift(1).fillna(tesis["tas_basal"])
tesis["TAS_lag2"] = tesis.groupby("idPaciente")["TAS"].shift(2).fillna(tesis["tas_basal"])

In [143]:
from sklearn.preprocessing import PolynomialFeatures

columns = tesis.columns

poly = PolynomialFeatures(interaction_only=True, degree=2)
tesis = poly.fit_transform(tesis)
new_columns = poly.get_feature_names_out(columns)
new_columns = [column.replace(' ', '*') for column in new_columns]

tesis = pd.DataFrame(tesis, columns=new_columns)

tesis['Intercept'] = 1

In [144]:
def model(data, effects):
    mixed = sm.MixedLM(endog=data['TAS'], exog=data[effects], exog_re=data[['Intercept', "Mes"]], groups=data['idPaciente']).fit(reml=False)
    return mixed

def model_gee(data, effects):
    mixed = sm.GEE(endog=data['TAS'], exog=data[effects], exog_re=data[['Intercept', "Mes"]], groups=data['idPaciente']).fit()
    print(mixed.summary())
    return mixed

### Modelo base

In [145]:
fixed_effects = [
    "Intercept",
    "Sexo",
    "Edad",
    "Mes",
]

### Probar efectos aleatorios

In [163]:
mixed_both = sm.MixedLM(
    endog=tesis["TAS"],
    exog=tesis[["Intercept", "Sexo", "Edad", "Adherencia", "Mes", "Adherencia*Mes"]],
    groups=tesis["idPaciente"],
    exog_re=tesis[["Intercept", "Mes"]]
).fit(reml=True)

print(mixed_both.summary())

              Mixed Linear Model Regression Results
Model:                MixedLM   Dependent Variable:   TAS        
No. Observations:     3920      Method:               REML       
No. Groups:           560       Scale:                113.2865   
Min. group size:      7         Log-Likelihood:       -15393.9298
Max. group size:      7         Converged:            Yes        
Mean group size:      7.0                                        
-----------------------------------------------------------------
                     Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
-----------------------------------------------------------------
Intercept           122.271    2.508 48.751 0.000 117.355 127.186
Sexo                  3.760    0.749  5.022 0.000   2.292   5.227
Edad                  0.167    0.039  4.344 0.000   0.092   0.243
Adherencia           -1.525    1.144 -1.333 0.183  -3.767   0.717
Mes                   0.371    0.240  1.545 0.122  -0.100   0.841
Adherencia*Mes       -0.

In [175]:
mixed_time = sm.MixedLM.from_formula(
    "TAS ~ Sexo + Edad + Adherencia + Mes + Adherencia*Mes",
    data=tesis,
    groups="idPaciente",
    vc_formula={"Mes": "0 + Mes"}
).fit(reml=True, maxiter=100000)

print(mixed_time.summary())
print(-2*mixed_time.llf, -2*mixed_both.llf, (-2*mixed_time.llf)-(-2*mixed_both.llf))

c:\Users\esteb\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\esteb\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2201: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
c:\Users\esteb\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\regression\mixed_linear_model.py:1534: RuntimeWarning: divide by zero encountered in log
  cov_aug_logdet = cov_re_logdet + np.sum(np.log(vc_var))
c:\Users\esteb\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\regression\mixed_linear_model.py:1538: RuntimeWarning: divide by zero encountered in divide
  solver = _smw_solver(1., ex_r, ex2_r, cov_re_inv, 1 / vc_var)
c:\Users\esteb\AppData\Local\Programs\Python\Python310\lib\site-packag

            Mixed Linear Model Regression Results
Model:              MixedLM  Dependent Variable:  TAS        
No. Observations:   3920     Method:              REML       
No. Groups:         560      Scale:               119.6644   
Min. group size:    7        Log-Likelihood:      -17097.4715
Max. group size:    7        Converged:           Yes        
Mean group size:    7.0                                      
-------------------------------------------------------------
                Coef.   Std.Err.   z    P>|z|  [0.025  0.975]
-------------------------------------------------------------
Intercept       123.998    2.510 49.393 0.000 119.077 128.918
Sexo              3.876    0.783  4.951 0.000   2.341   5.410
Edad              0.136    0.040  3.374 0.001   0.057   0.215
Adherencia       -1.442    1.032 -1.398 0.162  -3.464   0.580
Mes               0.355    1.866  0.191 0.849  -3.301   4.012
Adherencia:Mes   -0.775    0.279 -2.779 0.005  -1.322  -0.229
Mes Var        1914.

c:\Users\esteb\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


In [159]:
mixed_intercept = sm.MixedLM(
    endog=tesis["TAS"],
    exog=tesis[["Intercept", "Sexo", "Edad", "Adherencia", "Mes", "Adherencia*Mes"]],
    groups=tesis["idPaciente"],
    exog_re=tesis["Intercept"]
).fit(reml=True)

print(mixed_intercept.summary())
print(-2*mixed_intercept.llf, -2*mixed_both.llf, (-2*mixed_intercept.llf)-(-2*mixed_both.llf))

           Mixed Linear Model Regression Results
Model:             MixedLM  Dependent Variable:  TAS        
No. Observations:  3920     Method:              REML       
No. Groups:        560      Scale:               123.3531   
Min. group size:   7        Log-Likelihood:      -15419.4875
Max. group size:   7        Converged:           Yes        
Mean group size:   7.0                                      
------------------------------------------------------------
                Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
------------------------------------------------------------
Intercept      122.102    2.482 49.204 0.000 117.238 126.966
Sexo             3.771    0.752  5.016 0.000   2.297   5.244
Edad             0.165    0.039  4.261 0.000   0.089   0.241
Adherencia      -1.129    1.086 -1.039 0.299  -3.257   1.000
Mes              0.430    0.224  1.924 0.054  -0.008   0.868
Adherencia*Mes  -0.866    0.250 -3.467 0.001  -1.355  -0.376
Intercept Var   61.168    0.459     

### Selección de modelos

In [14]:
mixed = model(tesis, fixed_effects + ["Adherencia", "Adherencia*Mes"])
print(mixed.summary())
round(mixed.aic, 2), round(mixed.bic, 2)

              Mixed Linear Model Regression Results
Model:                MixedLM   Dependent Variable:   TAS        
No. Observations:     3920      Method:               ML         
No. Groups:           560       Scale:                113.2360   
Min. group size:      7         Log-Likelihood:       -15390.7343
Max. group size:      7         Converged:            Yes        
Mean group size:      7.0                                        
-----------------------------------------------------------------
                     Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
-----------------------------------------------------------------
Intercept           122.270    2.502 48.874 0.000 117.366 127.173
Sexo                  3.760    0.747  5.036 0.000   2.297   5.223
Edad                  0.167    0.038  4.357 0.000   0.092   0.243
Mes                   0.371    0.240  1.547 0.122  -0.099   0.841
Adherencia           -1.524    1.143 -1.333 0.183  -3.764   0.717
Adherencia*Mes       -0.

(30801.47, 30864.21)